In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")

## Connect

In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
       "X-Google-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

In [ ]:
# import weaviate, os

# # Connect to a Weaviate Cloud instance
# client = weaviate.connect_to_wcs(
#     cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),
#     headers={
#         "X-Palm-Api-Key": AI_STUDIO_KEY,
#     }
# )

# client.is_ready()

## Preview data

In [ ]:
questions = client.collections.get("Questions")

response = questions.query.fetch_objects(
    limit=5
)

for item in response.objects:
    print("question:", item.properties["question"])
    print("answer:  ", item.properties["answer"])
    print("points:  ", item.properties["points"])
    print("category:", item.properties["category"])
    print("round:   ", item.properties["round"])
    print("airDate: ", item.properties["airDate"], '\n')

## Vector search

In [ ]:
questions = client.collections.get("Questions")

response = questions.query.near_text(
    query="TV characters",
    limit=5
)

for item in response.objects:
    print(item.properties["question"])
    print(item.properties["answer"])
    print(item.properties["category"], "\n")

In [ ]:
questions = client.collections.get("Questions")

response = questions.query.hybrid(
    query="TV characters",
    limit=5
)

for item in response.objects:
    print(item.properties["question"])
    print(item.properties["answer"])
    print(item.properties["category"], "\n")

## Filters

In [ ]:
from weaviate.classes.query import Filter

wiki = client.collections.get("Questions")

response = wiki.query.fetch_objects(
    filters=Filter.by_property("category").like("TELEVISION"),
    limit=5
)

for item in response.objects:
    print(item.properties)

### Filter by date

* Use `datetime`:
```
datetime(2000, 1, 1).replace(tzinfo=timezone.utc)
```

* or formatted string:
```
"2000-01-01T00:00:00+00:00"
```

In [ ]:
from datetime import datetime, timezone
from weaviate.classes.query import Filter

questions = client.collections.get("Questions")

response = questions.query.fetch_objects(
    limit=5,
    filters= # in the year 2000
        Filter.by_property("airDate").greater_or_equal(datetime(2000, 1, 1).replace(tzinfo=timezone.utc)) &
        Filter.by_property("airDate").less_than(datetime(2001, 1, 1).replace(tzinfo=timezone.utc))
        
    # filters=
    #     Filter.by_property("airDate").greater_or_equal("2000-01-01T00:00:00+00:00") &
    #     Filter.by_property("airDate").less_than("2001-01-01T00:00:00+00:00")
)

for item in response.objects:
    print(item.properties)

## Close the client when done

In [ ]:
client.close()